In [4]:
#helper functions for caeser cipher

def ascii_encode(msg):

    lst = []
    for i in msg:
        lst.append(ord(i))

    return lst

def shift_ascii_numbers(numb_list, shift_val):
    '''
    we will restrict ourselves to the set of single printable characters from the ascii table. These range from 32 to 126 in the ascii table - a total of 95 characters. 
    
    To ensure no overflow after shifting, we first subtract 32 to take us to the range 0 to 94, make the shift modulo 95, then add 32 again to take us back to the suitable range. 
    '''

    shifted_numb_list =  []
    for i in numb_list:
        temp_val = ( (i -32 + shift_val) ) % 95 + 32
        shifted_numb_list.append(temp_val)
    return shifted_numb_list

def ascii_decode(msg):
    
    lst = []
    for i in msg:
        lst.append( chr(i))
        
    return lst


def ascii_caesar_encrypt(message, key):
    
    #step 1: ascii encode
    ascii_message = ascii_encode(message)
    
    #step 2: step 2: shift ascii numbers
    shifted_ascii_message = shift_ascii_numbers(ascii_message, key)
    
    #step 3: ascii decode 
    encrypted_message = ascii_decode(shifted_ascii_message)
    
    #step 4: convert to string and return 
    return ''.join(encrypted_message)

def ascii_caesar_decrypt(encrypted_message, key):
    
    #step 1: ascii encode
    ascii_message = ascii_encode(encrypted_message)
    
    #step 2: step 2: shift ascii numbers
    shifted_ascii_message = shift_ascii_numbers(ascii_message, -1*key)
    
    #step 3: ascii decode 
    decrypted_message = ascii_decode(shifted_ascii_message)
    
    #step 4: convert to string and return 
    return ''.join(decrypted_message)


In [5]:
#helper functions to get user input

def get_user_encryption_type():

    encryption_type = 0
    while (encryption_type not in ['1', '2']):
        encryption_type = input('Which encryption method would you like to use?\n (1. Ceasar cipher 2. RSA): ')
    encryption_type = int(encryption_type)

    return encryption_type


def get_user_message(message_limit = 500):
    message = input('Enter your message: ')
    while len(message) > message_limit:
        print('Your message is too long!')
        message = input('Enter your message: ')

    return message

def get_caesar_encryption_key():
    encryption_key = 0
    while (encryption_key not in [str(i) for i in range(1, 95)]):
        encryption_key = input('Enter an encryption key between 1 - 95: ')
    encryption_key = int(encryption_key)
    return encryption_key

def get_rsa_public_key():

    n = int(input("Enter the value of n from your public key:"))
    e = int(input("Enter the value of e from your public key:"))

    return public_key_from_variables(n, e)

def get_rsa_private_key():

    n = int(input("Enter the value of n from your private key:"))
    e = int(input("Enter the value of e from your private key:"))
    d = int(input("Enter the value of d from your private key:"))

    return private_key_from_variables(n, e, d)

def get_encryption_dir():
    encryption_dir = ''
    while encryption_dir not in ['e','d']: 
        encryption_dir = input("Would you like to encrypt (e) or decrypt (d)?")
    return encryption_dir

def encryption_keys_needed():
    keys_needed = input("Do you need to generate a public-private key pair? Y/N")
    
    if keys_needed == 'Y':
        return True
    else:
        return False


def output_message(message_str, encryption_dir):
    if encryption_dir == 'e':
        print('Your encrypted message is:')
    if encryption_dir == 'd':
        print('Your decrypted message is:')
    print(message_str)

def create_message_file(message_str, encryption_dir):
    if encryption_dir == 'e':
        filename = "encrypted_message.txt"
    elif encryption_dir == 'd':
        filename = "decrypted_message.txt"
        
    with open(filename, "w") as text_file:
        print(message_str, file=text_file)
        
def create_public_key_file(public_key_dict):
    n = public_key_dict['n']
    e = public_key_dict['e']
    
    print_string = "n = " + str(n) + "\n\ne = " + str(e)
    
    with open("public_key.txt", "w") as text_file:
        print(print_string, file=text_file)
        
def create_private_key_file(private_key_dict):
    n = private_key_dict['n']
    e = private_key_dict['e']
    d = private_key_dict['d']
    
    print_string = "n = " + str(n) + "\n\ne = " + str(e)+ "\n\nd = " + str(d)
    
    with open("private_key.txt", "w") as text_file:
        print(print_string, file=text_file)
    
    

    
    
    

In [6]:
#helper functions for RSA encryption 

#here we are following the methodology at 
#https://medium.com/coinmonks/rsa-encryption-and-decryption-with-pythons-pycryptodome-library-94f28a6a1816

# Import necessary modules from pycryptodome
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP

# This module converts binary data to hexadecimal
from binascii import hexlify, unhexlify




def create_RSA_key_pair(key_size = 1024):
    # Generate new RSA key
    # Create an RSA key pair with a key size of 1024 bits
    key = RSA.generate(key_size)
    
    # set the public_key variable to n and e as a dictionary 
    public_key_dict = {"n": key.n, "e":key.e}

    # Set the private_key variable to d
    private_key_dict = {"n": key.n, "e":key.e,"d": key.d}
    
    return private_key_dict, public_key_dict

def return_RSA_key_pair():
    #generate keys
    private_key_dict, public_key_dict = create_RSA_key_pair()
    #here we will print encryption keys to the user
    print('The details of your public key are:')
    print(f"\t n: {public_key_dict['n']}")
    print(f"\t e: {public_key_dict['e']}")
    print('The details of your private key are:')
    print(f"\t d: {private_key_dict['d']}")
    
    create_public_key_file(public_key_dict)
    create_private_key_file(private_key_dict)
    print('Files for public and private keys have been created!')


def public_key_from_variables(n, e):
    return RSA.construct((n, e))

def private_key_from_variables(n ,e ,d):
    return RSA.construct((n,e,d))

def RSA_encrypt(message_string, public_key_object):

    # Step 3: Encrypt using public key
    # Create a PKCS1_OAEP cipher object with the public key for encryption
    data_to_encrypt = str.encode(message_string)
    cipher_rsa = PKCS1_OAEP.new(public_key_object)

    #Encrypt the provided data using the public key
    encrypted = cipher_rsa.encrypt(data_to_encrypt)

    # Convert binary data to hexadecimal for display using hexlify
    return hexlify(encrypted).decode("utf-8")

def RSA_decrypt(encrypted_message_string, private_key_object):
    # Create a PKCS1_OAEP cipher object with the private key for decryption
    cipher_rsa = PKCS1_OAEP.new(private_key_object)
    decrypted = cipher_rsa.decrypt(unhexlify(encrypted_message_string))
    
    return decrypted.decode("utf-8")
    
#testing
#step 1: create keys as dicts
#private_key_dict, public_key_dict = create_RSA_key_pair()

#step2: encrypt using public key
#public_key_object = public_key_from_variables(public_key_dict['n'],public_key_dict['e'])
#encrypted_message = RSA_encrypt("hello world", public_key_object)
#print("Your encrypted message is:", encrypted_message.decode("utf-8"))

#step3: decrypt using private key
#private_key_object = private_key_from_variables(public_key_dict['n'],public_key_dict['e'], private_key_dict['d'])
#decrypted_message = RSA_decrypt(encrypted_message, private_key_object)
#print("Your decrypted message is:", decrypted_message)



    
    

In [9]:
# main logic 

while True:
    encryption_type = get_user_encryption_type()
    
    if encryption_type == 2:
        #USing RSA encryption. Does the user need to generate a key pair? 
        if encryption_keys_needed():
            return_RSA_key_pair()
 
            do_an_encryption = input("Would you also like to encrypt/decrypt a message? Y/N")
            if do_an_encryption == "N":
                break 
    
    message =  get_user_message()
    encryption_dir = get_encryption_dir()
    
    if encryption_type == 1:
        #the user wants to use the caeser cipher
        encryption_key = get_caesar_encryption_key()
        if encryption_dir == 'e':
            #the user wants to encrypt
            encrypted_str = ascii_caesar_encrypt(message, encryption_key)
            output_message(encrypted_str, encryption_dir)
            create_message_file(encrypted_str, encryption_dir)
        else:
            #the user wants to decrypt
            decrypted_str = ascii_caesar_decrypt(message, encryption_key)
            output_message(decrypted_str, encryption_dir)
            create_message_file(decrypted_str, encryption_dir)
        
    
    elif encryption_type == 2:
        #the user want to use RSA encryption
        
        if encryption_dir == 'e':
            #the user wants to encrypt
            public_key_obj = get_rsa_public_key()
            encrypted_str = RSA_encrypt(message, public_key_obj)
            print('Your encrypted message: ', encrypted_str)
            
        elif encryption_dir == 'd':
            
            #the user wants to decrypt
            private_key_obj = get_rsa_private_key()
            decrypted_str = RSA_decrypt(message,private_key_obj)
            print('Your decrypted message: ', decrypted_str)
    
    another_message = input("Would you like to encrypt/decrypt another message? Y/N")
    if another_message == "N":
        break
    

Your encrypted message is:
lipps


In [8]:
'''
    The details of your public key are:
	 n: 141676950560441764834351707860512105215923336298620809395878747611096953498162352179387554129116376739894260631802478524554005431980613976968467992284158106875854538950026804881064901222730340769687255915414705463878737252845039991988226802736995316679096341411733616197315922360882495976605565905005948109301
	 e: 65537
The details of your private key are:
	 d: 24333060034916650212482457599187089068929506589823700055846486459717523056827676520609523006505240346434072320545168351807069062397939956433115274137517484500550655269609316895353831783056674124232697141281834547879947107705636831127372227352457266259525148132910342453382008493509784641227823721795345598625




'''

'\n    The details of your public key are:\n\t n: 141676950560441764834351707860512105215923336298620809395878747611096953498162352179387554129116376739894260631802478524554005431980613976968467992284158106875854538950026804881064901222730340769687255915414705463878737252845039991988226802736995316679096341411733616197315922360882495976605565905005948109301\n\t e: 65537\nThe details of your private key are:\n\t d: 24333060034916650212482457599187089068929506589823700055846486459717523056827676520609523006505240346434072320545168351807069062397939956433115274137517484500550655269609316895353831783056674124232697141281834547879947107705636831127372227352457266259525148132910342453382008493509784641227823721795345598625\n\n\n\n\n'